In [4]:
import pandas as pd

# アメダスデータを用いた　よく使うpandas機能の整理
- 本　Notebookは 札幌のアメダスデータを用いて、Pandasでよく使ってる機能の整理を行う
- コードの綺麗さは追求しない
- pandasの機能にないのも一部含まれているが、私用のtips用に作成したものであるため、その点は目をつぶってほしい

In [56]:
# データの読み込み
df= pd.read_csv('./data/Sapporo_amedas.csv',skiprows=0,index_col=0,header=[1,2,4],encoding='cp932')##アメダスデータは特殊なencoding

In [57]:
df

札幌                                                \
年月日                   平均気温(℃)                   降水量の合計(mm)                    
           Unnamed: 1_level_2 品質情報 均質番号 Unnamed: 4_level_2 現象なし情報 品質情報 均質番号   
2020/1/1                 -5.8    8    1                2.5      0    8    1   
2020/1/2                 -4.0    8    1                0.0      0    8    1   
2020/1/3                 -2.5    8    1                0.5      0    8    1   
2020/1/4                 -5.0    8    1                1.5      0    8    1   
2020/1/5                 -4.7    8    1                2.0      0    8    1   
...                       ...  ...  ...                ...    ...  ...  ...   
2022/12/27                1.5    8    1                1.0      0    8    1   
2022/12/28                0.8    8    1                0.5      0    8    1   
2022/12/29               -2.3    8    1                4.0      0    8    1   
2022/12/30               -1.8    8    1                0.0      0    8    1   
2022/12/31                0.3    8    1                0.0      0    8    1   

                                         ...                      \
年月日             合計全天日射量(MJ/㎡)            ... 降雪量合計(cm)             
           Unnamed: 8_level_2 品質情報 均質番号  ...    現象なし情報 品質情報 均質番号   
2020/1/1                 3.44    8    1  ...         0    8    1   
2020/1/2                 4.63    8    1  ...         1    8    1   
2020/1/3                 5.36    8    1  ...         0    8    1   
2020/1/4                 7.32    8    1  ...         0    8    1   
2020/1/5                 4.51    8    1  ...         0    8    1   
...                       ...  ...  ...  ...       ...  ...  ...   
2022/12/27               4.62    8    1  ...         0    8    1   
2022/12/28               6.02    8    1  ...         0    8    1   
2022/12/29               4.14    8    1  ...         0    8    1   
2022/12/30               4.73    8    1  ...         1    8    1   
2022/12/31               4.79    8    1  ...         1    8    1   

                                                                               
年月日                平均現地気圧(hPa)                      日照時間(時間)                   
           Unnamed: 25_level_2 品質情報 均質番号 Unnamed: 28_level_2 現象なし情報 品質情報 均質番号  
2020/1/1                1009.3    8    1                 0.2      0    8    1  
2020/1/2                1011.1    8    1                 1.2      0    8    1  
2020/1/3                1008.6    8    1                 1.6      0    8    1  
2020/1/4                1010.4    8    1                 5.2      0    8    1  
2020/1/5                1015.0    8    1                 2.2      0    8    1  
...                        ...  ...  ...                 ...    ...  ...  ...  
2022/12/27              1016.5    8    1                 1.8      0    8    1  
2022/12/28              1013.8    8    1                 3.4      0    8    1  
2022/12/29              1009.7    8    1                 2.3      0    8    1  
2022/12/30              1015.2    8    1                 3.1      0    8    1  
2022/12/31              1014.9    8    1                 1.2      0    8    1  

[1096 rows x 31 columns]

## 前処理　part1 wide型データでテーブルデータを持たせる場合
- multi columnsの中に入っている　Unnamed:というものを除去した上で データクレンジングを行う処理

In [58]:
# Unnamedが邪魔なので消す関数の作成
def multi_columns(data_frame):
    df = data_frame.copy()
    if type(df.columns) is not pd.MultiIndex:
        return df

    # 列にUnnamedという文字の入った内容を削除しインデックスを振り直す
    df = df.rename(columns=lambda x: x if not 'Unnamed' in str(x) else '')
    df = df.reset_index()

    cols = df.columns
    copy_col = list(cols)

    # column.namesを新しいcolumnの一番上(インデックスセルの一番左)に持ってくる
    name_col = tuple([(name if name is not None else '') for name in cols.names])
    copy_col[0] = name_col

    # 無効セルは上に詰める
    for i, col in enumerate(copy_col):
        pack = [content for content in col if content != ""]
        copy_col[i] = tuple(pack + ([""] * (len(col)  - len(pack))))

    df.columns = pd.MultiIndex.from_tuples(copy_col)
    cols.names = tuple([None for x in cols.names])
    return df

In [59]:
df=multi_columns(df)

In [60]:
#カラム名の確認
df.columns

MultiIndex([('年月日',              '',       ''),
            ( '札幌',       '平均気温(℃)',       ''),
            ( '札幌',       '平均気温(℃)',   '品質情報'),
            ( '札幌',       '平均気温(℃)',   '均質番号'),
            ( '札幌',    '降水量の合計(mm)',       ''),
            ( '札幌',    '降水量の合計(mm)', '現象なし情報'),
            ( '札幌',    '降水量の合計(mm)',   '品質情報'),
            ( '札幌',    '降水量の合計(mm)',   '均質番号'),
            ( '札幌', '合計全天日射量(MJ/㎡)',       ''),
            ( '札幌', '合計全天日射量(MJ/㎡)',   '品質情報'),
            ( '札幌', '合計全天日射量(MJ/㎡)',   '均質番号'),
            ( '札幌',      '最深積雪(cm)',       ''),
            ( '札幌',      '最深積雪(cm)', '現象なし情報'),
            ( '札幌',      '最深積雪(cm)',   '品質情報'),
            ( '札幌',      '最深積雪(cm)',   '均質番号'),
            ( '札幌',     '平均風速(m/s)',       ''),
            ( '札幌',     '平均風速(m/s)',   '品質情報'),
            ( '札幌',     '平均風速(m/s)',   '均質番号'),
            ( '札幌',    '平均雲量(10分比)',       ''),
            ( '札幌',    '平均雲量(10分比)',   '品質情報'),
            ( '札幌',    '平均雲量(10分比)',   '

In [61]:
#日付をdatetime型に揃えて、
df.loc[:, ('年月日',              '',       '')] = pd.to_datetime(df[('年月日',              '',       '')])
df=df.set_index(('年月日',              '',       ''),inplace=False)

In [62]:
df

札幌                                                           \
           平均気温(℃)           降水量の合計(mm)                  合計全天日射量(MJ/㎡)        
                   品質情報 均質番号            現象なし情報 品質情報 均質番号               品質情報   
(年月日, , )                                                                     
2020-01-01    -5.8    8    1        2.5      0    8    1          3.44    8   
2020-01-02    -4.0    8    1        0.0      0    8    1          4.63    8   
2020-01-03    -2.5    8    1        0.5      0    8    1          5.36    8   
2020-01-04    -5.0    8    1        1.5      0    8    1          7.32    8   
2020-01-05    -4.7    8    1        2.0      0    8    1          4.51    8   
...            ...  ...  ...        ...    ...  ...  ...           ...  ...   
2022-12-27     1.5    8    1        1.0      0    8    1          4.62    8   
2022-12-28     0.8    8    1        0.5      0    8    1          6.02    8   
2022-12-29    -2.3    8    1        4.0      0    8    1          4.14    8   
2022-12-30    -1.8    8    1        0.0      0    8    1          4.73    8   
2022-12-31     0.3    8    1        0.0      0    8    1          4.79    8   

                 ...                                                     \
                 ... 降雪量合計(cm)           平均現地気圧(hPa)           日照時間(時間)   
           均質番号  ...    現象なし情報 品質情報 均質番号             品質情報 均質番号            
(年月日, , )        ...                                                      
2020-01-01    1  ...         0    8    1      1009.3    8    1      0.2   
2020-01-02    1  ...         1    8    1      1011.1    8    1      1.2   
2020-01-03    1  ...         0    8    1      1008.6    8    1      1.6   
2020-01-04    1  ...         0    8    1      1010.4    8    1      5.2   
2020-01-05    1  ...         0    8    1      1015.0    8    1      2.2   
...         ...  ...       ...  ...  ...         ...  ...  ...      ...   
2022-12-27    1  ...         0    8    1      1016.5    8    1      1.8   
2022-12-28    1  ...         0    8    1      1013.8    8    1      3.4   
2022-12-29    1  ...         0    8    1      1009.7    8    1      2.3   
2022-12-30    1  ...         1    8    1      1015.2    8    1      3.1   
2022-12-31    1  ...         1    8    1      1014.9    8    1      1.2   

                             
                             
           現象なし情報 品質情報 均質番号  
(年月日, , )                    
2020-01-01      0    8    1  
2020-01-02      0    8    1  
2020-01-03      0    8    1  
2020-01-04      0    8    1  
2020-01-05      0    8    1  
...           ...  ...  ...  
2022-12-27      0    8    1  
2022-12-28      0    8    1  
2022-12-29      0    8    1  
2022-12-30      0    8    1  
2022-12-31      0    8    1  

[1096 rows x 31 columns]

## 前処理　part２ long型データでテーブルデータを持たせる場合
- 後日作成予定

In [ ]:
## 